In [1]:
import pandas as pd
from math import sqrt
from pandarallel import pandarallel

In [4]:
def compute_ndvi(row):
    return (row.nir - row.red) / (row.nir + row.red)

def compute_savi(row, L=0.5):
    return 1 + L * (row.nir - row.red) / (row.nir + row.red + L)

def compute_evi(row, G=2.5, L=1, C1=6, C2=7.5):
    return G * (row.nir - row.red) / (row.nir + C1 * row.red - C2 * row.blue + L)

def compute_rep(row):
    rededge = (row.red + row.rededge3) / 2
    return 704 + 35 * (rededge - row.rededge1) / (row.rededge2 - row.rededge1)

def compute_osavi(row):
    return (row.nir - row.red) / (row.nir + row.red + 0.16)

def compute_rdvi(row):
    return (row.nir - row.red) / sqrt((row.nir + row.red))

def compute_mtvi1(row):
    return 1.2 * (1.2 * (row.nir - row.green) - 2.5 * (row.red - row.green))

def compute_lswi(row):
    return (row.nir - row.swir) / (row.nir + row.swir)

def compute_vi(path):
    df = pd.read_csv(path)
    bands = ['red', 'green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']
    df['ndvi'] = df.parallel_apply(compute_ndvi, axis=1)
    df['savi'] = df.parallel_apply(compute_savi, axis=1)
    df['evi'] = df.parallel_apply(compute_evi, axis=1)
    df['rep'] = df.parallel_apply(compute_rep, axis=1)
    df['osavi'] = df.parallel_apply(compute_osavi, axis=1)
    df['rdvi'] = df.parallel_apply(compute_rdvi, axis=1)
    df['mtvi1'] = df.parallel_apply(compute_mtvi1, axis=1)
    df['lswi'] = df.parallel_apply(compute_lswi, axis=1)
    df = df.drop(columns=bands)
    df.to_csv(f'{path.split(".csv")[0]}_vi.csv', index=False)
    return df

In [5]:
folder = 'fixed_0-00146'
pandarallel.initialize()

train_path = f'../../data/processed/{folder}/train.csv'
train_df = compute_vi(train_path)

test_path = f'../../data/processed/{folder}/test.csv'
test_df = compute_vi(test_path)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
